# Wetterdaten
- Reinigung
- Zusammenfügung in eine Jahres-Datei

## **Für jeden Ordner**:

In [ ]:
# ORDNER
import_directory = "WD_ungereinigt/2020/" # 2018, 2019, 2020 müssen jedes Mal geändert werden

In [ ]:
# DIR+DATEINAME
export_directory = "WD_gereinigt/test.csv" # oder enifach WD_gereinigt/test

In [ ]:
import pandas as pd
import numpy as np
import cufflinks as cf
from ipywidgets import interactive
import datetime as dt
import os


# SETTING PARAMETERS
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
# SETTING PARAMETERS
def open(filepath):
    return pd.read_csv(filepath, sep='\t', header=[0,1])

In [ ]:
def combine_timestamp_column(df):
    df[('Unnamed: 0_level_0', 'Date')] = pd.to_datetime(df[('Unnamed: 0_level_0', 'Date')], dayfirst=True)
    df[('Unnamed: 1_level_0', 'Time')] = pd.to_datetime(df[('Unnamed: 1_level_0', 'Time')])
    
    l_ts = []

    for i in range(0,  len(df["Unnamed: 1_level_0"]["Time"])):
        year = df["Unnamed: 0_level_0"]["Date"][i].year
        month = df["Unnamed: 0_level_0"]["Date"][i].month
        day = df["Unnamed: 0_level_0"]["Date"][i].day
        hour = df["Unnamed: 1_level_0"]["Time"][i].hour
        minute = df["Unnamed: 1_level_0"]["Time"][i].minute
    
        if 30 < minute < 60:
            minute = 30
        elif 0 < minute < 30:
            minute = 0

        l_ts.append(pd.Timestamp(year=year, month=month, day=day,  hour=hour, minute=minute))
        
    df["TimeStamp"] = l_ts
    return df

In [ ]:
def delete_columns(df):
    # Zu entfernende Spalten
    columns = ["Unnamed: 1_level_0", "Unnamed: 0_level_0", "Hi", "Low", "Out", "Dew", "Unnamed: 14_level_0", 
                 ("Heat", "D-D "), ("Heat", "Index"), ("Wind", "Chill"), ("Wind", "Run"), ("Wind", "Dir"), 
                 ("Cool", "D-D "), "Arc.", ("In ", "Temp"), ("In", "Hum"), ("In ", "Dew"), ("In ", "Heat")
              ]

    # Spalten entfernen
    df = df.drop(columns=columns)

    # Multiindex entfernen
    df.columns = df.columns.droplevel(level=1)
    # "Rain" Spalte richtig benennen
    df.rename(columns = {"Unnamed: 15_level_0":"Rain"}, inplace = True)

    # Nicht numerische Werte entfernen
    df["THW"] = pd.to_numeric(df["THW"], errors='coerce')
    
    
    return df

In [ ]:
def delete_rows(df):
    for i in range(0, len(df["TimeStamp"])):
        m_value = df["TimeStamp"][i].minute
        if m_value == 30:
            df = df.drop(index=i)
        
    return df.reset_index()

In [ ]:
def Reinige(filepath):
    df = open(filepath)
    df = combine_timestamp_column(df)
    df = delete_columns(df)
    df = delete_rows(df)
    df = df.drop(columns="index")
    df = df.set_index("TimeStamp")
    return df

In [ ]:
def Speichere(name, df):
    if name[-4:] != ".csv":
        name = name +'.csv'
    df.to_csv(name)
    return name

In [ ]:
frames = []
for filename in os.listdir(import_directory):
    print("PROCESSING: " + str(filename))
    df = Reinige(import_directory+filename)
    frames.append(df)
    print("APPENDED: " + str(filename))

In [ ]:
result = pd.concat(frames)

In [ ]:
Speichere(export_directory, result)